#Setup

In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip3 install transformers[torch]
!pip3 install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 902.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=6885adeca2453b954e6ca5e1fc29b90efecfb3f20ed9e57ee76cf658adb23474
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)


# Running the IndicNER Model

Let's try annotating some Indian language sentences and get the named entities

In [ ]:
# Import all the necessary classes and initialize the tokenizer and model.
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

In [ ]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)

    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [ ]:
# let us try with some example sentences here
sentence = 'लगातार हमलावर हो रहे शिवपाल और राजभर को सपा की दो टूक, चिट्ठी जारी कर कहा- जहां जाना चाहें जा सकते हैं'

predicted_labels = get_predictions(sentence=sentence,
                                   tokenizer=tokenizer,
                                   model=model
                                   )

for index in range(len(sentence.split(' '))):
  print( sentence.split(' ')[index] + '\t' + predicted_labels[index] )

लगातार	O
हमलावर	O
हो	O
रहे	O
शिवपाल	B-PER
और	O
राजभर	B-PER
को	O
सपा	B-ORG
की	O
दो	O
टूक,	O
चिट्ठी	O
जारी	O
कर	O
कहा-	O
जहां	O
जाना	O
चाहें	O
जा	O
सकते	O
हैं	O


In [ ]:
sentence = 'ಶರಣ್ ರ ನೀವು ನೋಡಲೇಬೇಕಾದ ಟಾಪ್ 5 ಕಾಮಿಡಿ ಚಲನಚಿತ್ರಗಳು'

predicted_labels = get_predictions(sentence=sentence,
                                   tokenizer=tokenizer,
                                   model=model
                                   )

for index in range(len(sentence.split(' '))):
  print( sentence.split(' ')[index] + '\t' + predicted_labels[index] )

ಶರಣ್	B-PER
ರ	O
ನೀವು	O
ನೋಡಲೇಬೇಕಾದ	O
ಟಾಪ್	O
5	O
ಕಾಮಿಡಿ	O
ಚಲನಚಿತ್ರಗಳು	O


#Naampadam Dataset

The _Naampadam_ Dataset is a large dataset for Named Entity Recognition in 11 Indian languages.  _Naampadam_ means "named entity" in Sanskrit.

In [ ]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode, DatasetDict
import random
lang='bn'


# Assuming raw is a DatasetDict
raw = load_dataset('ai4bharat/naamapadam', lang)

# Define the number of samples you want for each split
train_samples = 10266
validation_samples = 52
test_samples = 51

# Select a subset of the data for each split
raw_dataset = {
    'train': raw['train'].select(indices=list(range(train_samples))),
    'validation': raw['validation'].select(indices=list(range(validation_samples))),
    'test': raw['test'].select(indices=list(range(test_samples)))
}

# Access column names for each split
for split, data in raw_dataset.items():
    print(f"{split} columns: {data.column_names}")




# raw = load_dataset('ai4bharat/naamapadam', lang)['train']
# desired_size = 10266

# # Ensure that the desired size is not greater than the number of rows in the original dataset
# desired_size = min(desired_size, len(raw))

# # Select a random subset of the dataset
# raw_dataset = raw.shuffle(seed=42).select(list(range(desired_size)))

train columns: ['tokens', 'ner_tags']
validation columns: ['tokens', 'ner_tags']
test columns: ['tokens', 'ner_tags']


In [ ]:
# let's now print how the Dataset looks like
# raw_datasets
print(raw_dataset)

{'train': Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 10266
}), 'validation': Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 52
}), 'test': Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 51
})}


In [ ]:
raw_dataset['train'].column_names

['tokens', 'ner_tags']

In [ ]:
# let's print an instance of dataset
idx=1000
rec=raw_dataset['train'][idx]
for w, t in zip(rec['tokens'],rec['ner_tags']):
  print('{}\t{}'.format(w,t))


এরপর	0
,	0
তিনি	0
দেখেছিলেন	0
যে	0
,	0
মরিয়মের	1
এবং	0
তার	0
সঙ্গে	0
থাকা	0
শোকার্ত	0
ব্যক্তিদের	0
সঙ্গে	0
কথা	0
বলার	0
সময়	0
,	0
যিশু	1
গভীরভাবে	0
প্রভাবিত	0
হয়েছিলেন	0
।	0


In [ ]:
column_names = raw_dataset["train"].column_names
print(column_names)

features = raw_dataset["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [ ]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [ ]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


# Training an NER Model with the dataset

We have already seen how to get predictions from fine-tuned NER model. We will now use the pre-trained IndicBERT model and fine-tune it for NER task.

Let us download a pre-trained model and fine-tune it for the task of NER. We will have to use the `AutoModelForTokenClassification` class to fine-tune the model

**Load Pre-trained Model**

In [ ]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

NameError: name 'num_labels' is not defined

In [ ]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

model=model.to("cuda")

**Tokenize all texts and align the labels with them**

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
train_dataset = raw_dataset["train"]
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

In [ ]:
eval_dataset = raw_dataset["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

**Create Data Collator, Metrics**

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

<ipython-input-19-435f98c90f62>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


**Set Training Arguments**

In [ ]:
args=TrainingArguments(output_dir='output_dir',max_steps=5)
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,)





**Training**

In [ ]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

In [ ]:
trainer.args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_always_push=False,
hub_model

In [ ]:
train_result = trainer.train()
metrics = train_result.metrics

You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.551900
1000,0.405400
1500,0.350800
2000,0.301400
2500,0.295000
3000,0.244500


Step,Training Loss
500,0.551900
1000,0.405400
1500,0.350800
2000,0.301400
2500,0.295000
3000,0.244500
3500,0.230500


In [ ]:

metrics = trainer.evaluate()

trainer.log_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        3.0
  eval_LOC_f1             =      0.625
  eval_LOC_number         =         14
  eval_LOC_precision      =     0.5556
  eval_LOC_recall         =     0.7143
  eval_ORG_f1             =     0.4286
  eval_ORG_number         =         16
  eval_ORG_precision      =        0.5
  eval_ORG_recall         =      0.375
  eval_PER_f1             =     0.6494
  eval_PER_number         =         41
  eval_PER_precision      =     0.6944
  eval_PER_recall         =     0.6098
  eval_loss               =     0.3667
  eval_overall_accuracy   =     0.8994
  eval_overall_f1         =     0.5985
  eval_overall_precision  =     0.6212
  eval_overall_recall     =     0.5775
  eval_runtime            = 0:00:01.75
  eval_samples_per_second =     29.675
  eval_steps_per_second   =      3.995


# Multilingual Fine-Tuning

We now present a short tutorial to fine-tune the model on the combined data of all Indic languages

The _Naampadam_ Dataset is a large dataset for Named Entity Recognition in 11 Indian languages.  _Naampadam_ means "named entity" in Sanskrit.

In [3]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import load_dataset, DownloadMode, load_metric

# Languages to be used
languages = ['bn', 'or', 'as']

raw = {}

# Load dataset for each language
for lang in languages:
    raw[lang] = load_dataset('ai4bharat/naamapadam', lang,
                             use_auth_token='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',
                             download_mode=DownloadMode.FORCE_REDOWNLOAD)

# Define the number of samples you want for each split
train_samples = 10266
validation_samples = 52
test_samples = 51

# Select a subset of the data for each split for each language
raw_dataset = {}
for lang in languages:
    raw_dataset[lang] = {
        'train': raw[lang]['train'].select(indices=list(range(train_samples))),
        'validation': raw[lang]['validation'].select(indices=list(range(validation_samples))),
        'test': raw[lang]['test'].select(indices=list(range(test_samples)))
    }

# Access column names for each split for each language
for lang, splits in raw_dataset.items():
    print(f"Language: {lang}")
    for split, data in splits.items():
        print(f"\t{split} columns: {data.column_names}")


/usr/local/lib/python3.10/dist-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Generating train split:   0%|          | 0/961679 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/607 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4859 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/196793 [00:00<?, ? examples/s]

...

Generating validation split:   0%|          | 0/993 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/10266 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/51 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/52 [00:00<?, ? examples/s]

Language: bn
	train columns: ['tokens', 'ner_tags']
	validation columns: ['tokens', 'ner_tags']
	test columns: ['tokens', 'ner_tags']
Language: or
	train columns: ['tokens', 'ner_tags']
	validation columns: ['tokens', 'ner_tags']
	test columns: ['tokens', 'ner_tags']
Language: as
	train columns: ['tokens', 'ner_tags']
	validation columns: ['tokens', 'ner_tags']
	test columns: ['tokens', 'ner_tags']


We now concatenate all the datasets so that we could fine-tune a multilingual NER model

In [4]:
# Let's look at how raw looks like
raw_dataset

{'bn': {'train': Dataset({
      features: ['tokens', 'ner_tags'],
      num_rows: 10266
  }),
  'validation': Dataset({
      features: ['tokens', 'ner_tags'],
      num_rows: 52
  }),
  'test': Dataset({
      features: ['tokens', 'ner_tags'],
      num_rows: 51
  })},
 'or': {'train': Dataset({
      features: ['tokens', 'ner_tags'],
      num_rows: 10266
  }),
  'validation': Dataset({
      features: ['tokens', 'ner_tags'],
      num_rows: 52
  }),
  'test': Dataset({
      features: ['tokens', 'ner_tags'],
      num_rows: 51
  })},
 'as': {'train': Dataset({
      features: ['tokens', 'ner_tags'],
      num_rows: 10266
  }),
  'validation': Dataset({
      features: ['tokens', 'ner_tags'],
      num_rows: 52
  }),
  'test': Dataset({
      features: ['tokens', 'ner_tags'],
      num_rows: 51
  })}}

In [5]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_column_name = 'ner_tags'

label_list = raw_dataset['bn']["train"].features[label_column_name].feature.names
label_to_id = {label_list[i]: raw_dataset['bn']["train"].features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


To concatenate dataset let's combine all `train` and `validation` splits of all languages together

In [6]:
pre_concatenated_train_split = []

for lang in raw_dataset:
  pre_concatenated_train_split.append( raw_dataset[lang]['train'] )

pre_concatenated_validation_split = []

for lang in raw_dataset:
  pre_concatenated_validation_split.append( raw_dataset[lang]['validation'] )

Let us concatenate the dataset now

In [7]:
from datasets import concatenate_datasets, DatasetDict

concatenated_dataset = DatasetDict()
concatenated_dataset["train"] = concatenate_datasets(
    pre_concatenated_train_split
)

concatenated_dataset["validation"] = concatenate_datasets(
    pre_concatenated_validation_split
)

### Let us now load the Pre-trained model

In [8]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

model=model.to("cuda")

Tokenize all the datasets and align them

In [10]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
text_column_name = 'tokens'
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Let us not `tokenize` the `train` and `validation` splits

In [11]:
train_dataset = concatenated_dataset["train"]
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=32,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Running tokenizer on train dataset (num_proc=32):   0%|          | 0/30798 [00:00<?, ? examples/s]

In [12]:
validation_dataset = concatenated_dataset["validation"]
validation_dataset = validation_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on validation dataset",
)

Running tokenizer on validation dataset (num_proc=4):   0%|          | 0/156 [00:00<?, ? examples/s]

## Create DataCollator and Metrics

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [14]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

<ipython-input-14-435f98c90f62>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


## Set Training Arguments

In [15]:
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,)

In [16]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    args=args,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Train the model

In [17]:
# Train the model
train_result = trainer.train()
metrics = train_result.metrics

Step,Training Loss
500,0.544100
1000,0.451300
1500,0.378400
2000,0.391200
2500,0.370500
3000,0.339800
3500,0.337000


Step,Training Loss
500,0.544100
1000,0.451300
1500,0.378400
2000,0.391200
2500,0.370500
3000,0.339800
3500,0.337000


## Evaluate the Trained Model

Let us now evaluate the trained model on the test sets of all languages

We need to first tokenize the test sets

In [18]:
tokenized_test_set = {}

for lang in raw_dataset:
  tokenized_test_set[lang] = raw_dataset[lang]['test'].map(
      tokenize_and_align_labels,
      batched=True,
      num_proc=32,
      load_from_cache_file=True,
      desc="Running tokenizer on test dataset of language {0}".format(lang),
  )

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Running tokenizer on test dataset of language bn (num_proc=32):   0%|          | 0/51 [00:00<?, ? examples/s]

Running tokenizer on test dataset of language or (num_proc=32):   0%|          | 0/51 [00:00<?, ? examples/s]

Running tokenizer on test dataset of language as (num_proc=32):   0%|          | 0/51 [00:00<?, ? examples/s]

Run prediction on test set of each of the language separately and extract overall `Precison`, `Recall` and `F-Score` separately

In [19]:
final_metrics = {}

for lang in tokenized_test_set:
  predictions, labels, metrics = trainer.predict(tokenized_test_set[lang], metric_key_prefix=lang)

  lang_specific_results = {}
  for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall'] = metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1'] = metrics[key]
  final_metrics[lang] = lang_specific_results

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Print the individual result on each of the language

In [21]:
import pandas as pd

combined_results = pd.DataFrame.from_dict(
            final_metrics, orient="index"
        )

print(combined_results)
# metrics = trainer.evaluate()

# trainer.log_metrics("eval", metrics)

    Precision    Recall        F1
bn   0.465753  0.395349  0.427673
or   0.285714  0.076923  0.121212
as   0.428571  0.250000  0.315789


# Misc

In [ ]:
torch.__version__

In [ ]:
!nvidia-smi

In [ ]:
import torch
import gc
# del trainer
# del model
# del train_dataset
# del eval_dataset
# del data_collator
# del tokenizer
gc.collect()
torch.cuda.empty_cache()

from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)